In [6]:
import numpy as np
import pandas as pd
from transformers import pipeline
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from math import exp
from tqdm import tqdm

import nltk

nltk.download('punkt')

# Load Data
train_df = pd.read_csv("/content/train.csv").head(150)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# Select Pretrained Models
models = [
    "facebook/bart-large-cnn",
    "t5-large",
    "sshleifer/distilbart-cnn-12-6",
    "google/pegasus-large",
    "allenai/led-large-16384-arxiv",
]

# Example Semantic Coherence Metric Implementation
def semantic_coherence(generated_summary, dialogue):
    summary_tokens = word_tokenize(generated_summary.lower())
    dialogue_tokens = word_tokenize(dialogue.lower())

    common_tokens = set(summary_tokens) & set(dialogue_tokens)
    coherence_score = len(common_tokens) / len(summary_tokens)

    return coherence_score

# Example Factual Accuracy Metric Implementation
def factual_accuracy(generated_summary, reference_summary):
    gen_tokens = set(word_tokenize(generated_summary.lower()))
    ref_tokens = set(word_tokenize(reference_summary.lower()))

    common_tokens = gen_tokens & ref_tokens
    accuracy_score = len(common_tokens) / len(ref_tokens) if len(ref_tokens) != 0 else 0

    return accuracy_score

# Example Content Coverage Metric Implementation
def content_coverage(generated_summary, dialogue):
    summary_tokens = set(word_tokenize(generated_summary.lower()))
    dialogue_tokens = set(word_tokenize(dialogue.lower()))

    common_tokens = summary_tokens & dialogue_tokens
    coverage_score = len(common_tokens) / len(dialogue_tokens) if len(dialogue_tokens) != 0 else 0

    return coverage_score

# Initialize evaluation results DataFrame
evaluation_results = pd.DataFrame(columns=["Model", "BLEU Score", "Semantic Coherence", "Factual Accuracy", "Content Coverage"])

# Initialize empty list to store evaluation results
evaluation_results_list = []

# Apply Models and Evaluate
for model_name in models:
    print(f"Evaluating model: {model_name}")

    # Initialize the summarization pipeline
    summarizer = pipeline("summarization", model=model_name, tokenizer=model_name)

    # Initialize evaluation metric accumulators
    semantic_coherence_scores = []
    factual_accuracy_scores = []
    content_coverage_scores = []
    generated_summaries = []

    for index, row in tqdm(train_df.iterrows(), total=len(train_df)):
        dialogue, summary = row['dialogue'], row['summary']

        # Generate summaries
        generated_summary = summarizer(dialogue, max_length=100, min_length=30, do_sample=False)[0]["summary_text"]
        generated_summaries.append(generated_summary)

        # Evaluate Metrics
        semantic_coherence_scores.append(semantic_coherence(generated_summary, dialogue))
        factual_accuracy_scores.append(factual_accuracy(generated_summary, summary))
        content_coverage_scores.append(content_coverage(generated_summary, dialogue))

    # Calculate BLEU Score
    reference_summaries = train_df["summary"].tolist()
    bleu_score = corpus_bleu([[summary] for summary in reference_summaries], generated_summaries)

    # Append results to the evaluation results list
    evaluation_results_list.append({
        "Model": model_name,
        "BLEU Score": bleu_score,
        "Semantic Coherence": sum(semantic_coherence_scores) / len(semantic_coherence_scores),
        "Factual Accuracy": sum(factual_accuracy_scores) / len(factual_accuracy_scores),
        "Content Coverage": sum(content_coverage_scores) / len(content_coverage_scores),
    })

    # Print a separator for clarity
    print("=" * 50)

# Concatenate the evaluation results list into a DataFrame
evaluation_results = pd.concat([pd.DataFrame(item, index=[0]) for item in evaluation_results_list], ignore_index=True)

# Compare Results
print("BLEU Scores:", [result["BLEU Score"] for result in evaluation_results_list])

# Save evaluation results to a CSV file
evaluation_results.to_csv("evaluation_results.csv", index=False)


Evaluating model: facebook/bart-large-cnn


100%|██████████| 150/150 [20:35<00:00,  8.23s/it]


Evaluating model: t5-large


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

100%|██████████| 150/150 [54:29<00:00, 21.80s/it]

BLEU Scores: [0.3488121983156154, 0.3397876196809291, 0.30525945503925905, 0.36171531600300605, 0.15296891656854678]
